In [94]:
import pandas as pd
import dateutil.parser
import time
import functools
import collections
import itertools
from openai import RateLimitError

In [95]:
chars = ["D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+"]
limits = [float("-inf"), -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, float("inf")]
def get_char(haystack, char_set):
    for char in char_set:
        if haystack.find(char) != -1:
            return char
    return None
def map_decimal_to_bin(decimal, bins, limits):
    bins = ["D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+"]
    # Find the bin
    for i in range(len(limits) - 1):
        if limits[i] <= decimal < limits[i + 1]:
            return bins[i]
    return "Error: Decimal number does not fit any bin"

In [96]:
data = pd.read_pickle(".data/stock_data.pickle")
data


Price         Date       Close                                      \
Ticker                    AAPL        AMZN       GOOGL        MSFT   
0       2017-06-12   34.077194   48.245499   48.090500   64.347366   
1       2017-06-13   34.351360   49.039501   48.525002   65.149620   
2       2017-06-14   34.016258   48.823502   48.396500   64.799240   
3       2017-06-15   33.812382   48.208500   48.008999   64.458031   
4       2017-06-16   33.339035   49.385502   47.931000   64.550247   
...            ...         ...         ...         ...         ...   
1500    2023-05-26  174.739929  120.110001  124.610001  330.901917   
1501    2023-05-30  176.602585  121.660004  123.669998  329.231934   
1502    2023-05-31  176.552780  120.580002  122.870003  326.428802   
1503    2023-06-01  179.381607  122.769997  123.720001  330.593750   
1504    2023-06-02  180.238220  124.250000  124.669998  333.396912   

Price                             Dividends             ... Stock Splits       \
Ticker        ^GSPC          ^NDX      AAPL AMZN GOOGL  ...        GOOGL MSFT   
0       2429.389893   5708.180176       0.0  0.0   0.0  ...          0.0  0.0   
1       2440.350098   5751.819824       0.0  0.0   0.0  ...          0.0  0.0   
2       2437.919922   5727.069824       0.0  0.0   0.0  ...          0.0  0.0   
3       2432.459961   5700.890137       0.0  0.0   0.0  ...          0.0  0.0   
4       2433.149902   5681.479980       0.0  0.0   0.0  ...          0.0  0.0   
...             ...           ...       ...  ...   ...  ...          ...  ...   
1500    4205.450195  14298.410156       0.0  0.0   0.0  ...          0.0  0.0   
1501    4205.520020  14354.990234       0.0  0.0   0.0  ...          0.0  0.0   
1502    4179.830078  14254.089844       0.0  0.0   0.0  ...          0.0  0.0   
1503    4221.020020  14441.509766       0.0  0.0   0.0  ...          0.0  0.0   
1504    4282.370117  14546.639648       0.0  0.0   0.0  ...          0.0  0.0   

Price                 Volume                                             \
Ticker ^GSPC ^NDX       AAPL       AMZN     GOOGL      MSFT       ^GSPC   
0        0.0  0.0  289229200  188944000  84042000  47761700  4031840000   
1        0.0  0.0  136661600   91600000  40108000  25258600  3278240000   
2        0.0  0.0  126124800   79498000  30110000  25510700  3558640000   
3        0.0  0.0  128661600  107478000  48552000  26068700  3371980000   
4        0.0  0.0  201444400  229454000  51184000  48345100  5299380000   
...      ...  ...        ...        ...       ...       ...         ...   
1500     0.0  0.0   54835000   96779900  35635900  36630600  3715460000   
1501     0.0  0.0   55964400   64314800  35076700  29503100  4228510000   
1502     0.0  0.0   99625300   72800800  37325800  45950600  5980670000   
1503     0.0  0.0   68901800   54375100  30772700  26773900  4391860000   
1504     0.0  0.0   61945900   61215000  26963100  25864000  4454200000   

Price               
Ticker        ^NDX  
0       2588860000  
1       2063930000  
2       1949040000  
3       1870010000  
4       3133000000  
...            ...  
1500    4434070000  
1501    4843470000  
1502    5905760000  
1503    4610310000  
1504    4426760000  

[1505 rows x 43 columns]

In [97]:
#data = data.reset_index()
data['week'] = data['Date'].apply(lambda x: x.isocalendar()[1])
data['year'] = data['Date'].apply(lambda x: x.year)
data['day_of_week'] = data['Date'].apply(lambda x: x.isocalendar()[2])
data = data.set_index("Date")
data

Price            Close                                                   \
Ticker            AAPL        AMZN       GOOGL        MSFT        ^GSPC   
Date                                                                      
2017-06-12   34.077194   48.245499   48.090500   64.347366  2429.389893   
2017-06-13   34.351360   49.039501   48.525002   65.149620  2440.350098   
2017-06-14   34.016258   48.823502   48.396500   64.799240  2437.919922   
2017-06-15   33.812382   48.208500   48.008999   64.458031  2432.459961   
2017-06-16   33.339035   49.385502   47.931000   64.550247  2433.149902   
...                ...         ...         ...         ...          ...   
2023-05-26  174.739929  120.110001  124.610001  330.901917  4205.450195   
2023-05-30  176.602585  121.660004  123.669998  329.231934  4205.520020   
2023-05-31  176.552780  120.580002  122.870003  326.428802  4179.830078   
2023-06-01  179.381607  122.769997  123.720001  330.593750  4221.020020   
2023-06-02  180.238220  124.250000  124.669998  333.396912  4282.370117   

Price                    Dividends                  ... Stock Splits  \
Ticker              ^NDX      AAPL AMZN GOOGL MSFT  ...         ^NDX   
Date                                                ...                
2017-06-12   5708.180176       0.0  0.0   0.0  0.0  ...          0.0   
2017-06-13   5751.819824       0.0  0.0   0.0  0.0  ...          0.0   
2017-06-14   5727.069824       0.0  0.0   0.0  0.0  ...          0.0   
2017-06-15   5700.890137       0.0  0.0   0.0  0.0  ...          0.0   
2017-06-16   5681.479980       0.0  0.0   0.0  0.0  ...          0.0   
...                  ...       ...  ...   ...  ...  ...          ...   
2023-05-26  14298.410156       0.0  0.0   0.0  0.0  ...          0.0   
2023-05-30  14354.990234       0.0  0.0   0.0  0.0  ...          0.0   
2023-05-31  14254.089844       0.0  0.0   0.0  0.0  ...          0.0   
2023-06-01  14441.509766       0.0  0.0   0.0  0.0  ...          0.0   
2023-06-02  14546.639648       0.0  0.0   0.0  0.0  ...          0.0   

Price          Volume                                                         \
Ticker           AAPL       AMZN     GOOGL      MSFT       ^GSPC        ^NDX   
Date                                                                           
2017-06-12  289229200  188944000  84042000  47761700  4031840000  2588860000   
2017-06-13  136661600   91600000  40108000  25258600  3278240000  2063930000   
2017-06-14  126124800   79498000  30110000  25510700  3558640000  1949040000   
2017-06-15  128661600  107478000  48552000  26068700  3371980000  1870010000   
2017-06-16  201444400  229454000  51184000  48345100  5299380000  3133000000   
...               ...        ...       ...       ...         ...         ...   
2023-05-26   54835000   96779900  35635900  36630600  3715460000  4434070000   
2023-05-30   55964400   64314800  35076700  29503100  4228510000  4843470000   
2023-05-31   99625300   72800800  37325800  45950600  5980670000  5905760000   
2023-06-01   68901800   54375100  30772700  26773900  4391860000  4610310000   
2023-06-02   61945900   61215000  26963100  25864000  4454200000  4426760000   

Price      week  year day_of_week  
Ticker                             
Date                               
2017-06-12   24  2017           1  
2017-06-13   24  2017           2  
2017-06-14   24  2017           3  
2017-06-15   24  2017           4  
2017-06-16   24  2017           5  
...         ...   ...         ...  
2023-05-26   21  2023           5  
2023-05-30   22  2023           2  
2023-05-31   22  2023           3  
2023-06-01   22  2023           4  
2023-06-02   22  2023           5  

[1505 rows x 45 columns]

In [98]:
daily_open_close = data.groupby(by=["year","week","day_of_week"]).mean()[["Close","Open"]]
open_close = daily_open_close.reset_index("day_of_week")
grouped = open_close.groupby(by=["year","week"])
open_close["start_of_week"] = grouped.min()[["day_of_week"]]
open_close["end_of_week"] = grouped.max()[["day_of_week"]]
open_close.sort_values(by = ["year","week","day_of_week"])

Price     day_of_week       Close                                      \
Ticker                       AAPL        AMZN       GOOGL        MSFT   
year week                                                               
2017 24             1   34.077194   48.245499   48.090500   64.347366   
     24             2   34.351360   49.039501   48.525002   65.149620   
     24             3   34.016258   48.823502   48.396500   64.799240   
     24             4   33.812382   48.208500   48.008999   64.458031   
     24             5   33.339035   49.385502   47.931000   64.550247   
...               ...         ...         ...         ...         ...   
2023 21             5  174.739929  120.110001  124.610001  330.901917   
     22             2  176.602585  121.660004  123.669998  329.231934   
     22             3  176.552780  120.580002  122.870003  326.428802   
     22             4  179.381607  122.769997  123.720001  330.593750   
     22             5  180.238220  124.250000  124.669998  333.396912   

Price                                       Open                          \
Ticker           ^GSPC          ^NDX        AAPL        AMZN       GOOGL   
year week                                                                  
2017 24    2429.389893   5708.180176   34.152184   48.349998   47.936001   
     24    2440.350098   5751.819824   34.484934   48.899502   48.602001   
     24    2437.919922   5727.069824   34.564604   49.429501   48.775002   
     24    2432.459961   5700.890137   33.585079   47.935001   47.401001   
     24    2433.149902   5681.479980   33.692882   49.799999   47.895500   
...                ...           ...         ...         ...         ...   
2023 21    4205.450195  14298.410156  172.638244  116.040001  123.169998   
     22    4205.520020  14354.990234  176.263926  122.370003  125.639999   
     22    4179.830078  14254.089844  176.632467  121.449997  122.750000   
     22    4221.020020  14441.509766  177.001009  120.690002  122.820000   
     22    4282.370117  14546.639648  180.317907  124.919998  123.989998   

Price                                            start_of_week end_of_week  
Ticker           MSFT        ^GSPC          ^NDX                            
year week                                                                   
2017 24     63.858630  2425.879883   5677.160156             1           5  
     24     64.568664  2434.149902   5740.209961             1           5  
     24     65.389420  2443.750000   5774.120117             1           5  
     24     63.877074  2424.139893   5661.359863             1           5  
     24     64.301271  2431.239990   5694.209961             1           5  
...               ...          ...           ...           ...         ...  
2023 21    322.084865  4156.160156  13990.549805             1           5  
     22    333.227945  4226.709961  14465.589844             2           5  
     22    330.305505  4190.740234  14293.559570             2           5  
     22    323.983471  4183.029785  14269.089844             2           5  
     22    332.253786  4241.009766  14517.080078             2           5  

[1505 rows x 15 columns]

In [108]:
week_open = open_close[open_close["day_of_week"] == open_close[["start_of_week","end_of_week"]]["start_of_week"]][["Open",]]
week_close = open_close[open_close["day_of_week"] == open_close[["start_of_week","end_of_week"]]["end_of_week"]][["Close",]]
week = week_open.join(week_close)
weekly_tokens = (100*(week["Close"]-week["Open"]))/week["Open"]
weekly_tokens = weekly_tokens.map(lambda x: map_decimal_to_bin(x, chars, limits))
weekly_tokens.columns = pd.MultiIndex.from_product([["Real Token"], weekly_tokens.columns])
week = week.join(weekly_tokens)

In [109]:
news = pd.read_pickle(".data/summarized_weekly_news.pickle")
news

gpt_results  \
year week Topic                                                      
2017 23   GOOGL  Summary: The stock market saw new highs in com...   
     24   AAPL   Summary: AAPL shares have continued to fall in...   
          AMZN   Summary: The Amazon rainforest is at risk due ...   
          GOOGL  Summary: Microsoft has entered the laptop mark...   
          MSFT   Summary: Price target hit for Microsoft (NASDA...   
...                                                            ...   
2023 22   AMZN   Summary: Nearly 2,000 Amazon workers are plann...   
          GOOGL  Summary: Alphabet Inc. (GOOGL) is making signi...   
          MSFT   Summary: Microsoft stock has been in the spotl...   
          ^GSPC  Summary: The stock market was mixed on Tuesday...   
          ^NDX   Summary: The relationship between major stock ...   

                                                     Weeks Summary  
year week Topic                                                     
2017 23   GOOGL  Summary: The stock market reached record highs...  
     24   AAPL   Summary: AAPL shares have been falling due to ...  
          AMZN   Summary: Amazon has acquired Whole Foods Marke...  
          GOOGL  Summary: Microsoft enters laptop market with S...  
          MSFT   Summary: Analysts suggest short opportunities ...  
...                                                            ...  
2023 22   AMZN   Summary: The news covers various events relate...  
          GOOGL  Summary: The news data discusses the advanceme...  
          MSFT   Summary: The news highlights the recent develo...  
          ^GSPC  Summary: The stock market was mixed as investo...  
          ^NDX   Summary: The news data highlights a Volatility...  

[1477 rows x 2 columns]

In [110]:
joined_data = news.reset_index("Topic").pivot(values=["Weeks Summary"], columns=["Topic"]).join(week).sort_index()
joined_data

Weeks Summary  \
                                                        AAPL   
year week                                                      
2017 23                                                  NaN   
     24    Summary: AAPL shares have been falling due to ...   
     25    Summary: The tech sector is performing well bu...   
     26    Summary: Pandora is facing revenue struggles a...   
     27    Summary: The news data covers the review of th...   
...                                                      ...   
2023 18    Summary: The news data discusses recent events...   
     19    Summary: Warren Buffett praises Apple's strong...   
     20    Summary: Apple stock has seen a rise on NASDAQ...   
     21    Summary: The news data highlights the impact o...   
     22    Summary: The news data analyzes the importance...   

                                                              \
                                                        AMZN   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Amazon has acquired Whole Foods Marke...   
     25    Summary: Amazon's recent acquisition of Whole ...   
     26    Summary: The news discusses Amazon's competiti...   
     27    Summary: News data covers Amazon's recent acqu...   
...                                                      ...   
2023 18    Summary: Amazon and O'Reilly Automotive are bo...   
     19    Summary: The news data mentions the valuation ...   
     20    Summary: Arch Capital Management highlighted A...   
     21    Summary: The article discusses Amazon.com stoc...   
     22    Summary: The news covers various events relate...   

                                                              \
                                                       GOOGL   
year week                                                      
2017 23    Summary: The stock market reached record highs...   
     24    Summary: Microsoft enters laptop market with S...   
     25    Summary: The news data discusses the decline o...   
     26    Summary: Apple and Google are competing tech c...   
     27    Summary: Red Oak Technology Select Fund manage...   
...                                                      ...   
2023 18    Summary: Alphabet (GOOGL) is facing multiple s...   
     19    Summary: Alphabet (GOOGL) stock is being close...   
     20    Summary: Alphabet's stock has shown significan...   
     21    Summary: The article discusses the performance...   
     22    Summary: The news data discusses the advanceme...   

                                                              \
                                                        MSFT   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Analysts suggest short opportunities ...   
     25    Summary: Microsoft has released the Modern Key...   
     26    Summary: The news data covers Microsoft's fina...   
     27    Summary: High school graduates in the US are p...   
...                                                      ...   
2023 18    Summary: Vulcan Value Partners released a posi...   
     19    Summary: Diamond Hill Capital's Large Cap Stra...   
     20    Summary: Morningstar provides a positive analy...   
     21    Summary: The news data highlights a potential ...   
     22    Summary: The news highlights the recent develo...   

                                                              \
                                                       ^GSPC   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Jeff Immelt's departure as CEO of GE ...   
     25    Summary: Petronet LNG Ltd is in talks to buy a...   
     26    Summary: ONGC is planning to utilize GSPC's un...   
     27    

In [111]:
print(joined_data.iloc[1]["Weeks Summary"]["AAPL"])

Summary: AAPL shares have been falling due to a tech market slump and various factors like a white paper on Nvidia and political uncertainty in the UK. Additionally, a Bloomberg report on iPhone 8's data speed has impacted Apple stock. However, Goldman Sachs predicts that the tech sector stock rally may end soon, but companies with strong balance sheets like Apple, Microsoft, Facebook, Alphabet, Cisco Systems, and 3M are highlighted as good investments.

Keywords: AAPL shares, tech market, white paper, Nvidia, political uncertainty, UK, Bloomberg, iPhone 8, data speed, Goldman Sachs, tech sector, strong balance sheets, Apple, Microsoft, Facebook, Alphabet, Cisco Systems, 3M, dividend yield, cash holdings, investments, dividend growth, payout ratio.


In [122]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import langchain
gpt = ChatOpenAI(model="gpt-3.5-turbo")
company_profile = """Description: Apple Inc. (AAPL) is an American multinational technology company that specializes in consumer electronics, software, and services. It is best known for its iPhone smartphones, iPad tablets, Mac computers, and services like the App Store, Apple Music, and iCloud. Apple's emphasis on design and innovation has made it one of the most valuable companies in the world.

Positive Factors:
- **Innovation and Product Launches**: Continuous innovation and the release of new products can drive consumer interest and sales.
- **Brand Loyalty**: Strong brand recognition and a loyal customer base can sustain revenue and profit margins.
- **Services Growth**: Expansion of Apple's services sector, including streaming, financial services, and cloud storage, diversifies revenue streams.
- **Global Expansion**: Entry into emerging markets can provide new revenue sources.
- **Macroeconomic Trends**: A strong economic environment, with high consumer spending and low unemployment rates, can increase demand for Apple's products.

Negative Factors:
- **Competition**: Intense competition from other tech giants and emerging players can impact market share and pricing.
- **Supply Chain Disruptions**: Reliance on global supply chains makes Apple vulnerable to disruptions, tariffs, and geopolitical tensions.
- **Regulatory Challenges**: Increased scrutiny and regulation in key markets, including antitrust investigations and privacy regulations, can affect operations and costs.
- **Market Saturation**: High penetration rates in key markets can slow growth, making it challenging to maintain high sales volumes.
- **Macroeconomic Downturns**: Economic downturns, high inflation rates, or rising interest rates can reduce consumer spending on non-essential goods, impacting sales."""

news_summary_prompt = ChatPromptTemplate.from_template(f"""Instruction: Forecast next week stock return (price change) for symbol, given the company profile, historical weekly news summary, keywords, and stock returns, and optionally the examples from other stocks of a similar company. Return a single stock return.
The trend is represented by bins "D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+", where "D5+" means price dropping more than 5%, D5 means price dropping between 4% and 5%, "D4" means price dropping between 3% and 4%, "U5+" means price rising more than 5%, "U5" means price rising between 4% and 5%, "D4" means price rising between 3% and 4%, etc. Company Profile: {company_profile}
Recent News: News are ordered from oldest news to latest news. 
Example:
Company News:
{{example_company_news}}
Macreconomic News:
{{example_macro_news}}
Performance:
{{example_performance}}"

Actual Company News:
{{company_news}}
Macroeconomic News:
{{macro_news}}
""")


news_summary_chain = news_summary_prompt | gpt | StrOutputParser()
get_result = lambda p: news_summary_chain.invoke(p)
gpt_results = get_result({
    "company_news": joined_data["Weeks Summary"]["AAPL"].iloc[-1],
    "macro_news": joined_data["Weeks Summary"]["^NDX"].iloc[-1],
    "example_company_news": joined_data["Weeks Summary"]["AMZN"].iloc[0],
    "example_macro_news": joined_data["Weeks Summary"]["^NDX"].iloc[0],
    "example_performance": joined_data["Real Token"]["AMZN"].iloc[1],
})
print(gpt_results)

Based on the positive factors such as innovation and product launches, brand loyalty, services growth, global expansion, and macroeconomic trends, as well as the recent news discussing concerns about economic slowdown, iPhone demand issues, and inflation affecting operating expenses, I forecast that Apple Inc. (AAPL) stock will have a positive return next week. Therefore, I predict a stock return of "U3".


In [124]:
def with_n_history(iterable, n):
    "Returns the data along with a history of n records from the iterator"
    # with_n_history('ABCDEFG', 4) --> A AB ABC ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(maxlen=n)
    for x in it:
        window.append(x)
        yield tuple(window)
def batched(iter, batch_size):
    temp = []
    for i in iter:
        temp.append(i)
        if len(temp) == batch_size:
            yield temp
            temp = []
    yield temp

In [131]:
import time

def column_apply_with_saving(table: pd.DataFrame, write_column, fn , file_to_save = None):

    if write_column not in table:
        table[write_column] = ""

    for article_index in (table[table[write_column] == ""].index):
        row_data = table.loc[article_index]
        for i in range(5):
            try:
                result = fn(row_data)
                table.loc[article_index, write_column] = result
                print(f"{article_index}: {result}")
                if file_to_save is not None:
                    table.to_pickle(file_to_save)
                break
            except RateLimitError:
                time.sleep(20)
                continue
            except Exception as e:
                print(e)
                assert False

def get_overall_prediction (row):
    get_result = lambda p: news_summary_chain.invoke(p)
    prompt = {
        "company_news": row["Weeks Summary"]["AAPL"],
        "macro_news": row["Weeks Summary"]["^NDX"],
        "example_company_news": joined_data["Weeks Summary"]["AMZN"].iloc[1],
        "example_macro_news": joined_data["Weeks Summary"]["^NDX"].iloc[1],
        "example_performance": joined_data["Real Token"]["AMZN"].iloc[2],
    }
    gpt_results = get_result(prompt)
    return gpt_results
column_apply_with_saving(joined_data, "Final Prediction Response", get_overall_prediction, ".data/final_response.pickle")


(2023, 2): Based on the company profile and recent news, as well as the historical performance of the stock being in the "D2" trend, it is forecasted that Apple Inc. (AAPL) stock will experience a price drop of between 3% and 4% next week.
(2023, 3): Based on the positive factors such as innovation, brand loyalty, services growth, global expansion, and macroeconomic trends, as well as the recent recognition of a vice president at AAPL for leadership in the energy industry, it is likely that Apple Inc. (AAPL) will experience a positive stock return next week. Additionally, the discussion of exploring NDX options trading strategies based on the expected price reaction to the upcoming release of the Producer Price Index (PPI) suggests potential positive movement in the stock price. Therefore, the forecasted stock return for AAPL next week is "U2".
(2023, 4): Based on the company profile and recent news, as well as the historical performance trend of "D2", it is forecasted that Apple Inc. 

In [133]:
joined_data

Weeks Summary  \
                                                        AAPL   
year week                                                      
2017 23                                                  NaN   
     24    Summary: AAPL shares have been falling due to ...   
     25    Summary: The tech sector is performing well bu...   
     26    Summary: Pandora is facing revenue struggles a...   
     27    Summary: The news data covers the review of th...   
...                                                      ...   
2023 18    Summary: The news data discusses recent events...   
     19    Summary: Warren Buffett praises Apple's strong...   
     20    Summary: Apple stock has seen a rise on NASDAQ...   
     21    Summary: The news data highlights the impact o...   
     22    Summary: The news data analyzes the importance...   

                                                              \
                                                        AMZN   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Amazon has acquired Whole Foods Marke...   
     25    Summary: Amazon's recent acquisition of Whole ...   
     26    Summary: The news discusses Amazon's competiti...   
     27    Summary: News data covers Amazon's recent acqu...   
...                                                      ...   
2023 18    Summary: Amazon and O'Reilly Automotive are bo...   
     19    Summary: The news data mentions the valuation ...   
     20    Summary: Arch Capital Management highlighted A...   
     21    Summary: The article discusses Amazon.com stoc...   
     22    Summary: The news covers various events relate...   

                                                              \
                                                       GOOGL   
year week                                                      
2017 23    Summary: The stock market reached record highs...   
     24    Summary: Microsoft enters laptop market with S...   
     25    Summary: The news data discusses the decline o...   
     26    Summary: Apple and Google are competing tech c...   
     27    Summary: Red Oak Technology Select Fund manage...   
...                                                      ...   
2023 18    Summary: Alphabet (GOOGL) is facing multiple s...   
     19    Summary: Alphabet (GOOGL) stock is being close...   
     20    Summary: Alphabet's stock has shown significan...   
     21    Summary: The article discusses the performance...   
     22    Summary: The news data discusses the advanceme...   

                                                              \
                                                        MSFT   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Analysts suggest short opportunities ...   
     25    Summary: Microsoft has released the Modern Key...   
     26    Summary: The news data covers Microsoft's fina...   
     27    Summary: High school graduates in the US are p...   
...                                                      ...   
2023 18    Summary: Vulcan Value Partners released a posi...   
     19    Summary: Diamond Hill Capital's Large Cap Stra...   
     20    Summary: Morningstar provides a positive analy...   
     21    Summary: The news data highlights a potential ...   
     22    Summary: The news highlights the recent develo...   

                                                              \
                                                       ^GSPC   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Jeff Immelt's departure as CEO of GE ...   
     25    Summary: Petronet LNG Ltd is in talks to buy a...   
     26    Summary: ONGC is planning to utilize GSPC's un...   
     27    

In [134]:
joined_data["predicted_token"] = joined_data["Final Prediction Response"].apply(lambda x: get_char(x, chars))
joined_data

Weeks Summary  \
                                                        AAPL   
year week                                                      
2017 23                                                  NaN   
     24    Summary: AAPL shares have been falling due to ...   
     25    Summary: The tech sector is performing well bu...   
     26    Summary: Pandora is facing revenue struggles a...   
     27    Summary: The news data covers the review of th...   
...                                                      ...   
2023 18    Summary: The news data discusses recent events...   
     19    Summary: Warren Buffett praises Apple's strong...   
     20    Summary: Apple stock has seen a rise on NASDAQ...   
     21    Summary: The news data highlights the impact o...   
     22    Summary: The news data analyzes the importance...   

                                                              \
                                                        AMZN   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Amazon has acquired Whole Foods Marke...   
     25    Summary: Amazon's recent acquisition of Whole ...   
     26    Summary: The news discusses Amazon's competiti...   
     27    Summary: News data covers Amazon's recent acqu...   
...                                                      ...   
2023 18    Summary: Amazon and O'Reilly Automotive are bo...   
     19    Summary: The news data mentions the valuation ...   
     20    Summary: Arch Capital Management highlighted A...   
     21    Summary: The article discusses Amazon.com stoc...   
     22    Summary: The news covers various events relate...   

                                                              \
                                                       GOOGL   
year week                                                      
2017 23    Summary: The stock market reached record highs...   
     24    Summary: Microsoft enters laptop market with S...   
     25    Summary: The news data discusses the decline o...   
     26    Summary: Apple and Google are competing tech c...   
     27    Summary: Red Oak Technology Select Fund manage...   
...                                                      ...   
2023 18    Summary: Alphabet (GOOGL) is facing multiple s...   
     19    Summary: Alphabet (GOOGL) stock is being close...   
     20    Summary: Alphabet's stock has shown significan...   
     21    Summary: The article discusses the performance...   
     22    Summary: The news data discusses the advanceme...   

                                                              \
                                                        MSFT   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Analysts suggest short opportunities ...   
     25    Summary: Microsoft has released the Modern Key...   
     26    Summary: The news data covers Microsoft's fina...   
     27    Summary: High school graduates in the US are p...   
...                                                      ...   
2023 18    Summary: Vulcan Value Partners released a posi...   
     19    Summary: Diamond Hill Capital's Large Cap Stra...   
     20    Summary: Morningstar provides a positive analy...   
     21    Summary: The news data highlights a potential ...   
     22    Summary: The news highlights the recent develo...   

                                                              \
                                                       ^GSPC   
year week                                                      
2017 23                                                  NaN   
     24    Summary: Jeff Immelt's departure as CEO of GE ...   
     25    Summary: Petronet LNG Ltd is in talks to buy a...   
     26    Summary: ONGC is planning to utilize GSPC's un...   
     27    

In [135]:
import numpy as np
joined_data = joined_data[~(joined_data["Open"].isna() | joined_data["Open"].isna()).apply(lambda r: np.any(r), axis = 1)]

Bin Error

In [151]:
counts = np.unique((joined_data["Real Token"]["AAPL"] == joined_data["predicted_token"]), return_counts=True)[1]

In [152]:
100*counts[1]/(counts[0]+counts[1])

13.14102564102564

Binary Error

In [153]:
real = joined_data["Real Token"]["AAPL"].str.slice(0,1)
predicted = joined_data["predicted_token"].str.slice(0,1)
counts = np.unique((real == predicted), return_counts=True)[1]
100*counts[1]/(counts[0]+counts[1])

57.05128205128205

MSE Error

In [154]:
real = joined_data["Real Token"]["AAPL"].apply(lambda i: chars.index(i))
predicted = joined_data["predicted_token"].apply(lambda i: chars.index(i) if i != None else 0)
counts = np.unique((real == predicted), return_counts=True)[1]
100*counts[1]/(counts[0]+counts[1])

13.14102564102564

In [ ]:
from datetime import datetime
import base64
import shutil

# Get the current time
current_time = datetime.now().isoformat()

# Base64 encode the current time
encoded_time = base64.b64encode(current_time.encode()).decode()

# Define the .data folder path
data_folder_path = '.data/'

# Define the zip file name with the base64 encoded current time
zip_file_path = f'.checkpoints/{encoded_time}.zip'

# Zip the .data folder
shutil.make_archive(zip_file_path.rstrip('.zip'), 'zip', data_folder_path)

# Provide the encoded time and the zip file path
encoded_time, zip_file_path


('MjAyNC0wMy0wN1QxMTowODo1MS4zODAxNzI=',
 '.checkpoints/MjAyNC0wMy0wN1QxMTowODo1MS4zODAxNzI=.zip')